https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/open-MODIS-hdf4-files-python/


In [43]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'nyu', 'rbda_project_data'))


In [44]:
# Create a path to the vi_quality MODIS h4 data
vi_quality_path = os.path.join("vegetation_indices_all_time",
                               "vi_quality_unpacked",
                               "MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf")


In [45]:
# View dataset metadata
with rio.open(vi_quality_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta


<open DatasetReader name='vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf' mode='r'>


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [46]:
# Print all of the subdatasets in the data
with rio.open(vi_quality_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)


HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:0
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:1
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:2
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:3
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:4
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:5
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:6
HDF4_SDS:UNKNOWN:vegetation_indices_all_time/vi_quality_unpacked/MOD13A2.A2000049.h08v04.006.2015136104431_VI_quality.hdf:7
HDF4_SDS

In [47]:
# Create list to append arrays (of all type of data)
vi_quality_data = []

# Open the precipitation HDF5 file 
with rio.open(vi_quality_path) as dataset:
    
    # loop through each subdataset in HDF5 file
    for name in dataset.subdatasets:
        
        # Open the subdataset 
        with rio.open(name) as subdataset:
            modis_meta = subdataset.profile
            
            # Read data as a  2 dimensional array and append to list
            vi_quality_data.append(subdataset.read(1))


# vi_quality_stacked = np.stack(vi_quality_data)

In [48]:
# vi_quality_stacked

In [49]:
land_water_mask = vi_quality_data[6]
vi_usefulness = vi_quality_data[1]

In [50]:
land_water_mask

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ...,   1,   1,   1],
       [255, 255, 255, ...,   1,   1,   1],
       [255, 255, 255, ...,   1,   1,   1]], dtype=uint8)

In [51]:
vi_usefulness

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ...,   4,   3,   3],
       [255, 255, 255, ...,   3,   3,   0],
       [255, 255, 255, ...,   1,   0,   0]], dtype=uint8)

### Check for usefulness 

In [52]:
values = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
print(np.in1d(values, vi_usefulness))

[ True  True  True ... False False  True]


In [53]:
# print(land_water_mask)

In [54]:
# np.set_printoptions(threshold=8)

In [55]:
land_water_mask

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ...,   1,   1,   1],
       [255, 255, 255, ...,   1,   1,   1],
       [255, 255, 255, ...,   1,   1,   1]], dtype=uint8)

In [56]:
land_water_mask.shape

(1200, 1200)

## Extract only NDVI subdataset from first hdf file

In [57]:
# Create a path to the pre-fire MODIS h4 data
ndvi_path = os.path.join("vegetation_indices_all_time",
                               "MOD13A2.A2000049.h08v04.006.2015136104431.hdf")


In [58]:
# Create list to append arrays (of all type of data)
vegetation_data = []
vegetation_names = []

# Open the precipitation HDF5 file 
with rio.open(ndvi_path) as dataset:
    
    # loop through each subdataset in HDF5 file
    for name in dataset.subdatasets:
        
        # Open the subdataset 
        with rio.open(name) as subdataset:
            modis_meta = subdataset.profile
            
            # Read data as a  2 dimensional array and append to list
            vegetation_data.append(subdataset.read(1))
            vegetation_names.append(name);
#             np.savetxt(str(name)[-10:-1] + ".csv", subdataset.read(1), delimiter=",")

In [59]:
ndvi_data = vegetation_data[0]

In [60]:
ndvi_data

array([[-3000, -3000, -3000, ..., -3000, -3000, -3000],
       [-3000, -3000, -3000, ..., -3000, -3000, -3000],
       [-3000, -3000, -3000, ..., -3000, -3000, -3000],
       ...,
       [-3000, -3000, -3000, ...,  -165,  -174,    67],
       [-3000, -3000, -3000, ...,  -250,   244,  3151],
       [-3000, -3000, -3000, ...,  3432,  2938,  2648]], dtype=int16)